In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Dynamic_conv2d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        ratio=0.25,
        stride=1,
        padding=0,
        dilation=1,
        groups=1,
        bias=True,
        K=4,
        temperature=34,
        pool_mode="mix",
        init_weight=True,
    ):

In [ ]:
class channel_attention_module(nn.Module):
    def __init__(self,
                 in_channels,
                 ratio,
                 K,
                 temperature,
                 pool_mode="avg",
                 init_weight=True):

In [5]:
outs = []
outs.append(torch.Tensor(4,3,224,224))
outs.append(torch.Tensor(4,3,224,224))
torch.cat(outs, dim=1).shape

torch.Size([4, 6, 224, 224])

In [6]:
outs = []
outs.append(torch.Tensor(4,3,224,224))
torch.cat(outs, dim=1).shape

torch.Size([4, 3, 224, 224])

In [3]:
print(nn.AdaptiveAvgPool2d(1)(torch.randn(64,64,52,52)).shape)
print(nn.AdaptiveAvgPool2d(output_size=(1,1))(torch.randn(64,64,52,52)).shape)

torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])


In [2]:
class attention2d(nn.Module):
    def __init__(self, in_channels, ratio, K, temperature, init_weight=True):
        super().__init__()
        assert temperature % 3 == 1
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        if in_channels != 3:  # not RGB
            hidden_channels = int(in_channels * ratio) + 1
        else:
            hidden_channels = K
        self.fc1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=(1, 1),
            bias=False,
        )
        self.fc2 = nn.Conv2d(
            in_channels=hidden_channels, out_channels=K, kernel_size=(1, 1), bias=True
        )
        self.temperature = temperature
        if init_weight:  # default weight update
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    tensor=m.weight, mode="fan_out", nonlinearity="relu"
                )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            if isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def update_temperature(self):
        if self.temperature != 1:
            self.temperature -= 3
            print(f"change temperature to: {self.temperature}")

    def forward(self, x):
        x = self.avgpool(x)  # [B, in_channels, 1, 1]
        x = self.fc1(x)  # [B, in_channels*ratio+1, 1, 1]
        x = F.relu(x)
        x = self.fc2(x)  # [B, K, 1, 1]
        x = x.view(x.size(0), -1)  # [B, K]
        return F.softmax(x / self.temperature, 1)